# SK RAG Pattern Foundational Concepts - PgVector

Learning objectives:

- RAG pattern foundational concepts
- PostgreSQL PgVector

## Azure PostgreSQL Flexible Server - PGVector Setup in Azure

### Running a local database with a container:

- docker pull pgvector/pgvector:pg16
- Then execute:

```bash
docker run --name pgvector16 \
  --restart unless-stopped \
  -p 5432:5432 \
  -v pgdata:/var/lib/postgresql/data \
  -d pgvector/pgvector:pg16
```
- After deployment, connect using psql and type: `CREATE EXTENSION vector;`


### Running a Flexible Server in Azure - Manual Instructions:

- Create a Flexible server instance in the Azure Portal
- After creation, navigate to the Server Parameters pane:
  - Search for azure.extensions
  - Check the `Vector` value
  - Save the changes and wait for the server to deploy
- After deployment, open the instance and navigate to the `Database` panel:
  - Click `Connect` link on the Postgres database
    - Using the Cloud Shell psql, active the vector extension by typing: `CREATE EXTENSION vector;`

Connection string:
- `PG_CONN_STR="Host=<server>;Port=5432;Database=<database>;Username=<user>;Password=<password>"`


Useful commands:

- `truncate table public."LearningsCollection";`

## Setup

### Load required .NET packages and supporting constants, classes, etc.

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.4.0"
#r "nuget: Microsoft.SemanticKernel.Core, 1.4.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.4.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.Postgres, 1.4.0-alpha"
#r "nuget: Npgsql" 

#r "nuget: dotenv.net"

using System;

using System.IO;
using System.Text;
using System.Text.RegularExpressions;
using System.Text.Json;
using System.Text.Json.Serialization;
using Npgsql;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Connectors.Postgres;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Plugins.Memory;

using dotenv.net;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

#!import Models/Models.cs

const string MemoryCollectionName = "LearningsCollection";

Installed Packages dotenv.net, 3.1.3 Microsoft.SemanticKernel, 1.4.0 Microsoft.SemanticKernel.Connectors.Postgres, 1.4.0-alpha Microsoft.SemanticKernel.Core, 1.4.0 Microsoft.SemanticKernel.Plugins.Memory, 1.4.0-alpha Npgsql, 8.0.2

### Read the API Key and endpoints from environment variables or the .env file

In [2]:
// Load the .env file
DotEnv.Load();

// Get the OpenAI deployment name, endpoint, and key from the environment variables
var deploymentName = Environment.GetEnvironmentVariable("GPT_OPENAI_DEPLOYMENT_NAME");
var endpoint = Environment.GetEnvironmentVariable("GPT_OPENAI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("GPT_OPENAI_KEY");
var pg_conn_str = Environment.GetEnvironmentVariable("PG_CONN_STR");
var adaDeploymentName = "ada";

### Clear the PgVector embeddings table on every run

In [3]:
try 
{
    var conn = new NpgsqlConnection(pg_conn_str);
    conn.Open();
    var cmd = new NpgsqlCommand($"truncate table \"{MemoryCollectionName}\"", conn);
    Console.WriteLine($"{} table truncated");
    cmd.ExecuteNonQuery();
    conn.Close();
} catch {
    Console.WriteLine("Unable to truncate the table. It may not exist yest");
}

### Get a kernel instance configured for text completions and embeddings

In [4]:
// I'm using a RAM stored Vector DB, but I can switch providers like Azure Search, DuckDB, SQLite, etc.
#pragma warning disable CS8618,IDE0009,CA1051,CA1050,CA1707,CA2007,VSTHRD111,CS1591,RCS1110,CA5394,SKEXP0001,SKEXP0002,SKEXP0003,SKEXP0004,SKEXP0010,SKEXP0011,SKEXP0012,SKEXP0020,SKEXP0021,SKEXP0022,SKEXP0023,SKEXP0024,SKEXP0025,SKEXP0026,SKEXP0027,SKEXP0028,SKEXP0029,SKEXP0030,SKEXP0031,SKEXP0032,SKEXP0040,SKEXP0041,SKEXP0042,SKEXP0050,SKEXP0051,SKEXP0052,SKEXP0053,SKEXP0054,SKEXP0055,SKEXP0060,SKEXP0061,SKEXP0101,SKEXP0102

NpgsqlDataSourceBuilder dataSourceBuilder = new(pg_conn_str);
dataSourceBuilder.UseVector();
NpgsqlDataSource dataSource = dataSourceBuilder.Build();
IMemoryStore memoryStore = new PostgresMemoryStore(dataSource, vectorSize: 1536, schema: "public");


var kernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(deploymentName, endpoint, apiKey)
    .AddAzureOpenAITextEmbeddingGeneration(adaDeploymentName, endpoint, apiKey)
    .Build();


In [5]:
#pragma warning disable CS8618,IDE0009,CA1051,CA1050,CA1707,CA2007,VSTHRD111,CS1591,RCS1110,CA5394,SKEXP0001,SKEXP0002,SKEXP0003,SKEXP0004,SKEXP0010,SKEXP0011,SKEXP0012,SKEXP0020,SKEXP0021,SKEXP0022,SKEXP0023,SKEXP0024,SKEXP0025,SKEXP0026,SKEXP0027,SKEXP0028,SKEXP0029,SKEXP0030,SKEXP0031,SKEXP0032,SKEXP0040,SKEXP0041,SKEXP0042,SKEXP0050,SKEXP0051,SKEXP0052,SKEXP0053,SKEXP0054,SKEXP0055,SKEXP0060,SKEXP0061,SKEXP0101,SKEXP0102
var embeddingGenerator = new AzureOpenAITextEmbeddingGenerationService(adaDeploymentName, endpoint, apiKey);

// The combination of the text embedding generator and the memory store makes up the 'SemanticTextMemory' object used to
// store and retrieve memories.
SemanticTextMemory textMemory = new(memoryStore, embeddingGenerator);

## Ingestion

### Read the files and chunk them by paragraph

In [6]:
const string FOLDER_PATH = "data/";
string[] files = Directory.GetFiles(FOLDER_PATH);
List<Chunk> chunks = [];

foreach (string file in files)
{
    if (file.Contains("water"))
    {
        Console.WriteLine(file);        
        var contents = File.ReadAllText(file).Split("\n\n");
        var fileName = Path.GetFileName(file);
        var id = 1;
        foreach(var content in contents)
        {
            // Add the chunk to the list
            var c = new Chunk($"{fileName}-{contents.Length}-{id}",content, fileName);
            Console.WriteLine(c);
            chunks.Add(c);
            id++;
        }
    }
}

data/brittanica-water.txt
Chunk { Id = brittanica-water.txt-3-1, Text = Water, a substance composed of the chemical elements hydrogen and oxygen and existing in gaseous, liquid, and solid states. It is one of the most plentiful and essential of compounds. A tasteless and odourless liquid at room temperature, it has the important ability to dissolve many other substances. Indeed, the versatility of water as a solvent is essential to living organisms. Life is believed to have originated in the aqueous solutions of the world’s oceans, and living organisms depend on aqueous solutions, such as blood and digestive juices, for biological processes. Water also exists on other planets and moons both within and beyond the solar system. In small quantities water appears colourless, but water actually has an intrinsic blue colour caused by slight absorption of light at red wavelengths., File = brittanica-water.txt }
Chunk { Id = brittanica-water.txt-3-2, Text = Although the molecules of water are 

### Save every chunk as a memory

In [7]:
// Create an embedding generator to use for semantic memory.
foreach(var chunk in chunks)
{    
    await textMemory.SaveInformationAsync(MemoryCollectionName, id: chunk.Id, text: chunk.Text, description: chunk.File);
}

## Grounding

### Retrieve the memory based on a query

In [8]:
var question = "What is the chemical composition of water?";

#pragma warning disable CS8618,IDE0009,CA1051,CA1050,CA1707,CA2007,VSTHRD111,CS1591,RCS1110,CA5394,SKEXP0001,SKEXP0002,SKEXP0003,SKEXP0004,SKEXP0010,SKEXP0011,SKEXP0012,SKEXP0020,SKEXP0021,SKEXP0022,SKEXP0023,SKEXP0024,SKEXP0025,SKEXP0026,SKEXP0027,SKEXP0028,SKEXP0029,SKEXP0030,SKEXP0031,SKEXP0032,SKEXP0040,SKEXP0041,SKEXP0042,SKEXP0050,SKEXP0051,SKEXP0052,SKEXP0053,SKEXP0054,SKEXP0055,SKEXP0060,SKEXP0061,SKEXP0101,SKEXP0102
IAsyncEnumerable<MemoryQueryResult> queryResults =
                textMemory.SearchAsync(MemoryCollectionName, question, limit: 3, minRelevanceScore: 0.77);


### Find memories based on query, and collect the text in the memories to augment the prompt

In [9]:
// Keep the text for the recalled memories
StringBuilder context = new StringBuilder();

#pragma warning disable CS8618,IDE0009,CA1051,CA1050,CA1707,CA2007,VSTHRD111,CS1591,RCS1110,CA5394,SKEXP0001,SKEXP0002,SKEXP0003,SKEXP0004,SKEXP0010,SKEXP0011,SKEXP0012,SKEXP0020,SKEXP0021,SKEXP0022,SKEXP0023,SKEXP0024,SKEXP0025,SKEXP0026,SKEXP0027,SKEXP0028,SKEXP0029,SKEXP0030,SKEXP0031,SKEXP0032,SKEXP0040,SKEXP0041,SKEXP0042,SKEXP0050,SKEXP0051,SKEXP0052,SKEXP0053,SKEXP0054,SKEXP0055,SKEXP0060,SKEXP0061,SKEXP0101,SKEXP0102
await foreach (MemoryQueryResult r in queryResults)
{
    // Append the text
    context.Append("Text:\n"+r.Metadata.Text+"\n\n"+"Source:\n"+r.Metadata.Description+"\n\n");
}

// Final augmented text
var promptContext = context.ToString();
Console.WriteLine($"User:\n{question}\n\nNearest results:\n{promptContext}")

User:
What is the chemical composition of water?

Nearest results:
Text:
Water is an inorganic compound with the chemical formula H2O. It is a transparent, tasteless, odorless,[c] and nearly colorless chemical substance, and it is the main constituent of Earth's hydrosphere and the fluids of all known living organisms (in which it acts as a solvent[19]). It is vital for all known forms of life, despite not providing food energy or organic micronutrients. Its chemical formula, H2O, indicates that each of its molecules contains one oxygen and two hydrogen atoms, connected by covalent bonds. The hydrogen atoms are attached to the oxygen atom at an angle of 104.45°.[20] "Water" is also the name of the liquid state of H2O at standard temperature and pressure.

Source:
wikipedia-water.txt

Text:
Water, a substance composed of the chemical elements hydrogen and oxygen and existing in gaseous, liquid, and solid states. It is one of the most plentiful and essential of compounds. A tasteless and

## Process Prompt & Completion

### Create a SK function

In [10]:
const string promptTemplate = "{{$input}}\n\nContext: ===\n{{$context}}\n===\n\nadd a source reference to the end of each sentence. e.g. Apple is a fruit [reference1.pdf][reference2.pdf]. Use only the provided text.";
var excuseFunction = kernel.CreateFunctionFromPrompt(promptTemplate, new OpenAIPromptExecutionSettings() { MaxTokens = 500, Temperature = 0.4, TopP = 1 });


### Submit the prompt and print the results

In [11]:
string SplingLines(string longLine, int max_size =120) 
{
    int currentLineLength = 0;
    StringBuilder sb = new StringBuilder();
    foreach (string word in longLine.Split(' '))
    {
        if (currentLineLength + word.Length >= max_size)
        {
            sb.AppendLine();
            currentLineLength = 0;
        }

        sb.Append(word + " ");
        currentLineLength += word.Length + 1;
    }
    return sb.ToString();
}

In [12]:
var arguments = new KernelArguments()
        {
            ["input"] = question,
            ["context"] = promptContext
        };
var result = await kernel.InvokeAsync(excuseFunction, arguments);

Console.WriteLine($"user:\n{question}\n");
Console.WriteLine($"assistant:\n{SplingLines(result.ToString())}");

user:
What is the chemical composition of water?

assistant:
Water is an inorganic compound with the chemical formula H2O. It is a transparent, tasteless, odorless, and nearly 
colorless chemical substance, and it is the main constituent of Earth's hydrosphere and the fluids of all known living 
organisms (in which it acts as a solvent). It is vital for all known forms of life, despite not providing food energy 
or organic micronutrients. Its chemical formula, H2O, indicates that each of its molecules contains one oxygen and two 
hydrogen atoms, connected by covalent bonds. The hydrogen atoms are attached to the oxygen atom at an angle of 104.45°. 
"Water" is also the name of the liquid state of H2O at standard temperature and pressure. [source: 
wikipedia-water.txt]

Water, a substance composed of the chemical elements hydrogen and oxygen and existing in gaseous, 
liquid, and solid states. It is one of the most plentiful and essential of compounds. A tasteless and odourless liquid 
at